In [ ]:
import json
import os
import numpy as np
import tensorflow as tf
from pathlib2 import Path
import preprocess
from data_loader import create_dataset, create_inputs

In [ ]:
def prepare_data(data_dir, language):
    if language == "Chinese":
        train_path = data_dir / "train_cn.json"
        test_path = data_dir / "test_cn.json"
        dev_path = data_dir / "dev_cn.json"

    elif language == "English":
        train_path = data_dir / "train_en.json"
        test_path = data_dir / "test_en.json"
        dev_path = data_dir / "dev_en.json"

    else:
        raise ValueError("Language must be English or Chinese")

    if not os.path.isfile(test_path):
        test_path = None

    return train_path, dev_path, test_path

data_dir = "./dataset"
language = "Chinese"
train_path, dev_path, test_path = prepare_data(data_dir=Path(data_dir), language=language)
train_path, dev_path, test_path

In [ ]:
data  = create_inputs(json_path=train_path, plm="BERT",
                                                max_len=512,
                                                is_train=True, language="Chinese", task="nugget")
dataset = create_dataset(data, "nugget")
a = list(dataset.take(1))

In [ ]:
from model import create_dialogue_model
model = create_dialogue_model(plm_name="BERT", language="Chinese", max_turn_number=7, layer_num=1)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
ckpt_path = "./checkpoint/"

In [ ]:
callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_path, save_weights_only=True, verbose=1, monitor="val_rnss", save_best_only=True, mode="max")

In [ ]:
data2  = create_inputs(json_path=dev_path, plm="BERT",
                                                max_len=512,
                                                is_train=True, language="Chinese", task="nugget")
val_dataset = create_dataset(data2, "nugget", 10000)

In [ ]:
history = model.fit(x=dataset, epochs=3, verbose="auto",
                                                   callbacks=callback, validation_data=val_dataset, 
                    steps_per_epoch=128)